In [2]:
%load_ext autoreload
%autoreload 2

import torch
from circuits_benchmark.utils.get_cases import get_cases
import iit.model_pairs as mp
from circuits_benchmark.utils.ll_model_loader.ll_model_loader_factory import get_ll_model_loader
from circuits_benchmark.benchmark.benchmark_case import BenchmarkCase
import os

task_idx = '3'
out_dir = f'results/tuned_lens/{task_idx}'
os.makedirs(out_dir, exist_ok=True)

task: BenchmarkCase = get_cases(indices=[task_idx])[0]

ll_model_loader = get_ll_model_loader(task, interp_bench=True)
hl_ll_corr, model = ll_model_loader.load_ll_model_and_correspondence(device='cuda' if torch.cuda.is_available() else 'cpu')
# turn off grads
model.eval()
model.requires_grad_(False)

hl_model = task.get_hl_model()
model_pair = mp.StrictIITModelPair(hl_model, model, hl_ll_corr)

{'hook_embed': HookPoint(), 'hook_pos_embed': HookPoint(), 'blocks.0.attn.hook_k': HookPoint(), 'blocks.0.attn.hook_q': HookPoint(), 'blocks.0.attn.hook_v': HookPoint(), 'blocks.0.attn.hook_z': HookPoint(), 'blocks.0.attn.hook_attn_scores': HookPoint(), 'blocks.0.attn.hook_pattern': HookPoint(), 'blocks.0.attn.hook_result': HookPoint(), 'blocks.0.mlp.hook_pre': HookPoint(), 'blocks.0.mlp.hook_post': HookPoint(), 'blocks.0.hook_attn_in': HookPoint(), 'blocks.0.hook_q_input': HookPoint(), 'blocks.0.hook_k_input': HookPoint(), 'blocks.0.hook_v_input': HookPoint(), 'blocks.0.hook_mlp_in': HookPoint(), 'blocks.0.hook_attn_out': HookPoint(), 'blocks.0.hook_mlp_out': HookPoint(), 'blocks.0.hook_resid_pre': HookPoint(), 'blocks.0.hook_resid_mid': HookPoint(), 'blocks.0.hook_resid_post': HookPoint(), 'blocks.1.attn.hook_k': HookPoint(), 'blocks.1.attn.hook_q': HookPoint(), 'blocks.1.attn.hook_v': HookPoint(), 'blocks.1.attn.hook_z': HookPoint(), 'blocks.1.attn.hook_attn_scores': HookPoint(), 'b

In [3]:
max_len = 1000
unique_test_data = task.get_clean_data(max_samples=max_len, unique_data=True)

loader = torch.utils.data.DataLoader(unique_test_data, batch_size=256, shuffle=False, drop_last=False)

In [4]:
from interp_utils.node_stats import get_node_norm_stats

node_norms = get_node_norm_stats(model_pair, loader, return_cache_dict=False)

/Users/cybershiptrooper/src/interpretability/MATS/circuits-benchmark/notebooks/interp_utils/node_stats/node_norms.py:87: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  node_norms = pd.concat(


In [5]:
import circuits_benchmark.commands.evaluation.iit.iit_eval as eval_node_effect
max_len = 100
node_effects, eval_metrics = eval_node_effect.get_node_effects(case=task, model_pair=model_pair, use_mean_cache=False, max_len=max_len)

100%|██████████| 1/1 [00:00<00:00, 94.09it/s]


In [6]:
# combine node effects with node_norms
import pandas as pd
combined_df = pd.merge(node_effects, node_norms, left_on="node", right_on="name", how="inner")
combined_df.drop(columns=["name", "in_circuit"], inplace=True)
combined_df

,node,status,resample_ablate_effect,zero_ablate_effect,norm_cache,norm_std
0,"blocks.0.attn.hook_result, head 0",not_in_circuit,0.0,0.000,0.169730,0.033656
1,"blocks.0.attn.hook_result, head 1",not_in_circuit,0.0,0.000,0.137441,0.028349
2,"blocks.0.attn.hook_result, head 2",not_in_circuit,0.0,0.000,0.145937,0.010959
3,"blocks.0.attn.hook_result, head 3",not_in_circuit,0.0,0.000,0.078689,0.016923
4,"blocks.1.attn.hook_result, head 0",not_in_circuit,0.0,0.000,1.075704,0.091683
5,"blocks.1.attn.hook_result, head 1",not_in_circuit,0.0,0.000,0.363112,0.046042
6,"blocks.1.attn.hook_result, head 3",not_in_circuit,0.0,0.428,1.648610,0.047580
7,blocks.1.mlp.hook_post,not_in_circuit,0.0,0.000,1.269013,0.249035
8,blocks.0.mlp.hook_post,in_circuit,1.0,0.968,1.589742,0.103468
9,"blocks.1.attn.hook_result, head 2",in_circuit,1.0,0.720,3.892245,1.212605


In [7]:
import plotly.express as px

fig = px.scatter(combined_df, x="zero_ablate_effect", 
                 y="norm_cache", color="status",
                 error_y="norm_std",
                 # color map
                 color_discrete_map={
                    "in_circuit": "green",
                    "not_in_circuit": "orange",
                 },
                 labels={
                     "zero_ablate_effect": "Zero Ablation Effect",
                     "norm_cache": "Norm of Node Activation",
                     "status": "",
                     "resample_ablate_effect": "Resample Ablate Effect",
                 },
                 hover_data=["node", "resample_ablate_effect"],
                 # remove background grid and color
                 template="plotly_white",
                 )

# decrease margin
fig.update_layout(margin=dict(l=70, r=70, t=70, b=70))
# increase font size
fig.update_layout(font=dict(size=16))
fig.show()
# save to file as pdf
fig.write_image(f"node_stats_{task.get_name()}.pdf")


In [ ]:
from interp_utils.resample_ablate.collect_cache import collect_activations, collect_pca_directions

# collect activations
activations = collect_activations(model_pair, loader)
pca_dirs = collect_pca_directions(activations)

In [16]:
from iit.utils.node_picker import get_all_individual_nodes_in_circuit, find_ll_node_by_name

nodes = get_all_individual_nodes_in_circuit(model, hl_ll_corr)

# plot cosine similarity matrix for all nodes
import numpy as np

def get_cosine_similarity_matrix(pca_dirs, nodes_in_circuit, n_components=2):
    all_components_stacked = []
    for i in n_components:
        component_stacked = np.vstack([pca_dirs[node][i] for node in nodes_in_circuit])
        all_components_stacked.append(component_stacked)
    
    all_components_stacked = np.stack(all_components_stacked, axis=0)
    # now each component is a vector


In [22]:
for k, v in pca_dirs.items():
    print(k, v[0].shape)

LLNode(name='blocks.0.attn.hook_result', index=[:, :, 0, :], subspace=None) (2, 12)
LLNode(name='blocks.0.attn.hook_result', index=[:, :, 1, :], subspace=None) (2, 12)
LLNode(name='blocks.0.attn.hook_result', index=[:, :, 2, :], subspace=None) (2, 12)
LLNode(name='blocks.0.attn.hook_result', index=[:, :, 3, :], subspace=None) (2, 12)
LLNode(name='blocks.0.mlp.hook_post', index=[:], subspace=None) (2, 48)
LLNode(name='blocks.1.attn.hook_result', index=[:, :, 0, :], subspace=None) (2, 12)
LLNode(name='blocks.1.attn.hook_result', index=[:, :, 1, :], subspace=None) (2, 12)
LLNode(name='blocks.1.attn.hook_result', index=[:, :, 2, :], subspace=None) (2, 12)
LLNode(name='blocks.1.attn.hook_result', index=[:, :, 3, :], subspace=None) (2, 12)
LLNode(name='blocks.1.mlp.hook_post', index=[:], subspace=None) (2, 48)


In [21]:
pca_dirs.keys()

dict_keys([LLNode(name='blocks.0.attn.hook_result', index=[:, :, 0, :], subspace=None), LLNode(name='blocks.0.attn.hook_result', index=[:, :, 1, :], subspace=None), LLNode(name='blocks.0.attn.hook_result', index=[:, :, 2, :], subspace=None), LLNode(name='blocks.0.attn.hook_result', index=[:, :, 3, :], subspace=None), LLNode(name='blocks.0.mlp.hook_post', index=[:], subspace=None), LLNode(name='blocks.1.attn.hook_result', index=[:, :, 0, :], subspace=None), LLNode(name='blocks.1.attn.hook_result', index=[:, :, 1, :], subspace=None), LLNode(name='blocks.1.attn.hook_result', index=[:, :, 2, :], subspace=None), LLNode(name='blocks.1.attn.hook_result', index=[:, :, 3, :], subspace=None), LLNode(name='blocks.1.mlp.hook_post', index=[:], subspace=None)])